<a href="https://colab.research.google.com/github/iman-g/Pay-gap-Survey-kaggle/blob/main/pay_Gap_Survey_Kaggle_Statistic_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Table of contents**<a id='toc0_'></a>    
- [Data Cleansing](#toc1_)    
- [Model](#toc2_)    
  - [One Hot Encoding](#toc2_1_)    
  - [Simple OLS](#toc2_2_)    
  - [Top ten effective independent variables on salary](#toc2_3_)    
  - [Conclusion](#toc2_4_)    
  - [Examining the effect](#toc2_5_)    
  - [Lasso for regularization](#toc2_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import statsmodels.api as sm


pd.set_option('display.max_columns', 500)

# <a id='toc1_'></a>[Data Cleansing](#toc0_)

In [ ]:
df = pd.read_csv('kaggle_survey_2020_responses.csv')

df_final = df.iloc[1:]

/var/folders/n9/p7_5pdrx7c5bw5ybznkg0xb40000gn/T/ipykernel_52631/1413690633.py:1: DtypeWarning: Columns (0,18,31,45,50,63,80,92,99,105,130,171,187,241,254,266,278,289,307,322,330,342,353) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('kaggle_survey_2020_responses.csv')


In [ ]:
df = df_final.copy()

# Keeping man and woman
df = df[df['Q2'].isin(['Man','Woman'])]

# employed responders
df = df[~df['Q5'].isin(['Student','Currently not employed'])]


# salary extraction
df['salary'] = df.Q24.apply(lambda x: str(x).replace('$','').replace(',','').replace('>','').strip())
df['salary_avg'] = (df['salary'].apply(lambda x: 5000 if '-' not in x else int(x.split('-')[0])) +
                    df['salary'].apply(lambda x: 5000 if '-' not in x else int(x.split('-')[1])))/2


# Age
df['age'] = (df['Q1'].apply(lambda x: 70 if '-' not in x else int(x.split('-')[1])) +
                 df['Q1'].apply(lambda x: 70 if '-' not in x else int(x.split('-')[0])))/2

df.drop(['Time from Start to Finish (seconds)', 'Q24','salary', 'Q1'], axis =1, inplace = True)

# <a id='toc2_'></a>[Model](#toc0_)

## <a id='toc2_1_'></a>[One Hot Encoding](#toc0_)

In [ ]:
y = ['salary_avg']
df_y = df[y]
df_x = df.drop(y, axis = 1)

In [ ]:
df_x.dtypes.value_counts()

object     352
float64      1
Name: count, dtype: int64

In [ ]:
numeric_columns = df_x.select_dtypes(exclude=['object']).columns.values.tolist()
object_columns = df_x.select_dtypes(include=['object']).columns.values.tolist()

In [ ]:
def one_hot_encode(df, columns):

    encoder = OneHotEncoder(sparse_output=False, drop='first')

    encoded_data = encoder.fit_transform(df[columns])

    new_columns_list = []
    for col, values in zip(columns, encoder.categories_):
        for value in values[1:]:
            unique_values = df[col].dropna().unique()

            if len(unique_values) == 1:

                value = unique_values[0]
                new_col_name = f"{col}_{value}"
                df[new_col_name] = df[col].apply(lambda x: 1 if pd.notna(x) else 0)

                new_columns_list.append(new_col_name)
            else:

                transformed = encoder.fit_transform(df[[col]])
                encoded_col_names = [f"{col}_{cat}" for cat in encoder.categories_[0][1:]]
                new_columns_list.append(f"{col}_{value}")

    encoded_df = pd.DataFrame(encoded_data, columns=new_columns_list, index=df.index)

    df = pd.concat([df.drop(columns, axis=1), encoded_df], axis=1)

    return df


## <a id='toc2_2_'></a>[Simple OLS](#toc0_)

In [ ]:
new_df_x = one_hot_encode(df_x, object_columns)

model = sm.OLS(df_y['salary_avg'], X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             salary_avg   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     30.35
Date:                Mon, 07 Oct 2024   Prob (F-statistic):               0.00
Time:                        11:44:43   Log-Likelihood:            -1.5456e+05
No. Observations:               12968   AIC:                         3.101e+05
Df Residuals:                   12481   BIC:                         3.137e+05
Df Model:                         486                                         
Covariance Type:            nonrobust                                         
                                                                                                                                      coef    std err          t      P>|t|      [0.025      0.975]
--------------

## <a id='toc2_3_'></a>[Top ten effective independent variables on salary](#toc0_)

In [ ]:
coefficients = model.params
p_values = model.pvalues


results = pd.DataFrame({
    'Column Name': coefficients.index,
    'Coefficient': coefficients.values,
    'P-value': p_values.values
})

results.sort_values(by='P-value').head(10)

,Column Name,Coefficient,P-value
369,Q3_United States of America,67740.122707,2.941737e-67
541,"Q25_$100,000 or more ($USD)",29182.152816,1.498912e-37
361,Q3_Switzerland,76270.250255,6.070358e-33
335,Q3_Israel,52438.437737,6.431655e-20
540,"Q25_$10,000-$99,999",18124.429193,1.387742e-18
317,Q3_Australia,41732.627813,1.739195e-18
469,Q15_20 or more years,28904.576647,1.169473e-17
472,Q15_5-10 years,15552.785877,1.654520e-17
322,Q3_Canada,35313.018662,3.710462e-15
368,Q3_United Kingdom of Great Britain and Norther...,32933.484814,7.725986e-15


## <a id='toc2_4_'></a>[Conclusion](#toc0_)

In [ ]:

print(
f"""Model R-squered: {round(model.rsquared,3)}
Number of dependent variable: {len(X.columns)}
Coefficient Woman: {round(model.params['Q2_Woman'])}
P-Value of Woman coefficient: {round(model.pvalues['Q2_Woman'],4)}
Number of dependent variable with P-Value less than 0.05 (significantly affect salary): {model.params[model.pvalues < 0.05].count()
}"""
)


Model R-squered: 0.542
Number of dependent variable: 801
Coefficient Woman: -3192
P-Value of Woman coefficient: 0.0005
Number of dependent variable with P-Value less than 0.05 (significantly affect salary): 171


In [ ]:
print("""Using an OLS regression model, we isolated the impact of gender on salary. Our findings indicate that being a woman is associated with
 lower earnings. Subsequently, we analyzed the magnitude of this gender-based salary disparity.""")

Using an OLS regression model, we isolated the impact of gender on salary. Our findings indicate that being a woman is associated with
 lower earnings. Subsequently, we analyzed the magnitude of this gender-based salary disparity.


## <a id='toc2_5_'></a>[Examining the effect](#toc0_)

In [ ]:
df['log_salary'] = np.log(df_y['salary_avg'])


model2 = sm.OLS(df['log_salary'], X).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_salary   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                     27.47
Date:                Mon, 07 Oct 2024   Prob (F-statistic):               0.00
Time:                        11:37:46   Log-Likelihood:                -21541.
No. Observations:               12968   AIC:                         4.406e+04
Df Residuals:                   12481   BIC:                         4.769e+04
Df Model:                         486                                         
Covariance Type:            nonrobust                                         
                                                                                                                                      coef    std err          t      P>|t|      [0.025      0.975]
--------------

In [ ]:
print(
f"""Model R-squered: {round(model.rsquared,3)}
Coefficient Woman: {round(model.params['Q2_Woman'],2)}
P-Value of Woman coefficient: {round(model.pvalues['Q2_Woman'],4)}
"""
)


Model R-squered: 0.517
Coefficient Woman: -0.21
P-Value of Woman coefficient: 0.0



In [ ]:
print("""Being a woman is associated with a 21% decrease in salary.""")

Being a woman is associated with a 21% decrease in salary.


## <a id='toc2_6_'></a>[Lasso for regularization](#toc0_)

In [ ]:
results_reg = sm.OLS(df_y['salary_avg'], X).fit_regularized(L1_wt=1, alpha= 5)
final = sm.regression.linear_model.OLSResults(model,results_reg.params,model.normalized_cov_params)

In [ ]:
from sklearn.metrics import r2_score

y_pred = results_reg.predict(X)


mse = r2_score(df_y['salary_avg'], y_pred)
print(f'Mean Squared Error (MSE): {mse}')

Mean Squared Error (MSE): 0.538984561568473


In [ ]:
results_reg.params['Q2_Woman']

-3321.7050914171305